# Pumas.jl Workshop Solutions
### Chris Rackauckas, Vijay Ivaturi

# Problem 1: Simulate a first-order absorption model with linear elimination after a 100 mg oral dose in 24 subjects

Parameters are: `Ka = 1 hr-1`, `CL = 1 L/hr`, `V = 20 L/hr`.

## Part 1: Setup the population

In [ ]:
using Pumas, Plots, CSV, Random
Random.seed!(0)

In [ ]:
single_dose_regimen = DosageRegimen(100, time=0)
first(single_dose_regimen.data)

to build a sinlge subject

In [ ]:
s1 = Subject(id=1, evs=single_dose_regimen,cvs=(Wt=70,))

let's first define a function to choose body weight randomly

In [ ]:
choose_covariates() = (Wt = rand(55:80),)

Then, we use generate a population of subjects with a random weight generated from the covariate function above

In [ ]:
pop = Population(map(i -> Subject(id = i,evs = single_dose_regimen, cvs =  choose_covariates()),1:24))

You can view the generated population using by calling a random subject by index and look at
the subject's

* covariates
* events
* id numbers
* observations
* time

Let us us peek at the first subject's covariates

In [ ]:
pop[1].covariates

## Part 2: Write the model

In [ ]:
mymodel = @model begin
  @param   begin
    tvcl ∈ RealDomain(lower=0, init = 1.0)
    tvv ∈ RealDomain(lower=0, init = 20)
    tvka ∈ RealDomain(lower = 0, init= 1)
    Ω ∈ PDiagDomain(init=[0.09,0.09, 0.09])
    σ_prop ∈ RealDomain(lower=0,init=0.04)
  end

  @random begin
    η ~ MvNormal(Ω)
  end

  @pre begin
    CL = tvcl * (Wt/70)^0.75 * exp(η[1])
    V  = tvv * (Wt/70) * exp(η[2])
    Ka = tvka * exp(η[3])
  end
  @covariates Wt

  @dynamics Depots1Central1
    #@dynamics begin
    #    Depot' =  -Ka*Depot
    #    Central' =  Ka*Depot - (CL/V)*Central
    #end

  @derived begin
      cp = @. 1000*(Central / V)
      dv ~ @. Normal(cp, sqrt(cp^2*σ_prop))
    end
end

Note that above, we are using the analytical solution in `@dynamics`. You can switch to using the
differential equation system if you prefer.

## Part 3: Simulate

Let's first extract the model parameters

In [ ]:
param = init_param(mymodel)

Then using the `simobs` function, carry out the simulation and visualize the simulation output

In [ ]:
obs = simobs(mymodel, pop, param, obstimes=0:1:72)
plot(obs)

where
* `mymodel` is the model setup in the Part 2,
* `pop` is the population of subjects that was setup in Part 1
* `param` is the specified set of model parameters
* `obstimes` specifies the simulation time period.

# Problem 2: Peform Non-compartmental analysis

We will start by generating a dataframe of the resuls from the simulation step

In [ ]:
simdf = DataFrame(obs)
first(simdf, 6)

For the purpose of NCA, let us use the `cp` (output without residual error) as our observed value

To prepare the dataset for NCA analysis, let us use the `read_nca` function. The NCA datasets in Pumas requires
a `route` specification which can either be `iv` or `ev`. Since this is an oral drug administration, lets add that
to the `simdf`.

In [ ]:
simdf[!, :route] .= "ev"

Next we can define time, concentration and dose units so the report includes the units for the pharmacokinetic parameters.
The general syntax for units are `u` followed by the unit in quotes `""`.

In [ ]:
timeu = u"hr"
concu = u"mg/L"
amtu  = u"mg"

In [ ]:
ncadf = read_nca(simdf, id=:id, time=:time, conc=:cp, amt=:amt,
    route=:route,timeu=timeu, concu=concu, amtu=amtu, lloq=0.4concu)

You can view the concentration-time plots by doing

In [ ]:
plot(ncadf)

You can then generate `cmax` and `auc` for each subject

In [ ]:
auc = NCA.auc(ncadf)

In [ ]:
cmax = NCA.cmax(ncadf)

Or generate the entire NCA report using

In [ ]:
report = NCAReport(ncadf)
report = NCA.to_dataframe(report)
first(report,6)

# Problem 3: Estimate using Non-linear mixed effects

We can use the simulated dataset in the Problem 1 for our estimation. We need a couple of data manipulation steps

1. missing `cmt` should be converted to 2 to reflect central compartment
2. data rows where `time = 0`, and `cp=0` should be removed

In [ ]:
simdf.cmt = ifelse.(ismissing.(simdf.cmt), 2, simdf.cmt)
est_df = simdf[.!((simdf.dv .== 0.0) .& (simdf.cmt .==2)),:]
first(est_df,6)

## Part 1: Read datasets for NLME estimation

We can use the `read_pumas` function to prepare the dataset for NLME estimation

In [ ]:
data = read_pumas(est_df ,cvs = [:Wt], dvs=[:dv])

where

* `cvs` takes an array of covariates
* `dvs` takes an array of the dependent variables
* since the dataframe has `time` as the variable, the function does not need a specific input


## Part 2: Perform a model fit

We now use the
* `mymodel` model that we wrote earlier
* the set of parameters specified in `param` as initial estimates
* `data` that was read in using the `read_pumas` function

to fit the model.

In [ ]:
res = fit(mymodel,data,param,Pumas.FOCEI())

## Part 3: Infer the results

infer provides the model inference

In [ ]:
infer(res)

## Part 4: Inspect the results

inspect gives you the
* model predictions
* residuals
* Empirical Bayes estimates

In [ ]:
preds = DataFrame(predict(res))
first(preds, 6)

In [ ]:
resids = DataFrame(wresiduals(res))
first(resids, 6)

In [ ]:
ebes = DataFrame(empirical_bayes(res))
first(ebes, 6)

There is an `inspect` function that provides all the results at once

_Note that this function below fails to convert into a dataframe due to a bug. Will be fixed soon_

In [ ]:
resout = DataFrame(inspect(res))
first(resout, 6)